# Udacity Data Investigation Project

# Movies Dataset Investigation

### Purpose

Conduct data analysis on movie dataset and create a file to share that documents your findings on Udacity classroom, unit 2 Project: Investigate a Dataset.

### Introduction

This movie dataset was supplied by Udacity, called IMDb movie data:

[IMDb movie data](https://s3.amazonaws.com/video.udacity-data.com/topher/2018/July/5b57919a_data-set-options/data-set-options.pdf)

RangeIndex: 10866 entries, 0 to 10865

Data columns (total 21 columns):

dtypes: float64(4), int64(6), object(11)

### Process
This project uses Pandas, NumPy, Matplotlib, and other libraries to wrangle, explore, analyze and communicate data.

- 3 spearate .py files for functions used in the ipynb files

### Acknowledgments

- Stackoverflow
- Udacity lessons
- Data Validation Project
- GitHub/unionai-oss/pandera

### Hypothesis Questions
- What 5 year period were the most films released?
- What years were each genre at it's highest popularity?
- Are some genres more popular per 5 years? 
- What is the most common day of the week that a movie is released? 
- Do movies generate more revenue if they are released certain days or on a holiday?

## Movies dataset exploration

In [ ]:
# Import libraries
import pandas as pd
%run categories.py
%run validate.py
%run group.py

In [ ]:
# Read csv files into dataframe, view the first few rows
mdf=pd.read_csv('data/movies.csv')
mdf.head()

In [ ]:
mdf.tail()

In [ ]:
# View data types and shape, notice missing values
mdf.info()

In [ ]:
mdf.shape

In [ ]:
#Explore Data types
mdf.dtypes

In [ ]:
# View release date dtype
mdf['release_date'].head()

In [ ]:
# count unique years present in dataset
year_count= len(list(mdf['release_year'].unique()))
year_count

In [ ]:
# View totals with missing cast
no_cast=mdf[mdf['cast'].isna()]
no_cast.count()

In [ ]:
# Explore null values in dataset
null_col=mdf.isnull().sum()

In [ ]:
# list of all columns with null values
all_null_col= [k for k, v in null_col.items() if v >0]
all_null_col

In [ ]:
# Find duplicate rows
mdf[mdf.duplicated()]

In [ ]:
# find all columns with multiple values | separated
for c in list(mdf.columns):
    v=mdf[c].iloc[0]
    if isinstance(v, str) and '|' in v:
        newline = '/n'
        print(f'{c} column has multiple values:')
        print('--------------------------------')

In [ ]:
mdf['keywords'].unique()

In [ ]:
mdf['genres'].unique()

### Explore categories, popularity, revenue, and year

- separate categories for production companies and genres
- determine unique category names

In [ ]:
# get list of unique genre names
%run categories.py
mdf['genres']= replace_pipe(mdf, 'genres')
genre=get_unique_list(mdf, 'genres')
len(genre)

In [ ]:
# get list of unique production company names
mdf['production_companies']= replace_pipe(mdf, 'production_companies')
prod=get_unique_list(mdf, 'production_companies')
len(prod)

In [ ]:
# Revenue, budget
mdf['revenue'].describe()

In [ ]:
# Revenue, budget
mdf['revenue_adj'].describe()

In [ ]:
# Revenue, budget
mdf['popularity'].describe()

> END EXPLORE DATASET

## Dataset wrangling and validation

In [ ]:
from pandera import Column, DataFrameSchema
pd.options.mode.chained_assignment = None  # default='warn'
from pathlib import Path
import yaml
import black
import sys
import logging
import numpy as np
import pandera as pd
%run validate.py
%run categories.py

In [ ]:
# Create schema to keep track of columns of how I want data shaped
schema = pa.DataFrameSchema(
    {
    'popularity': Column(pa.Float64),
    'budget': Column(pa.Int64),
    'revenue': Column(pa.Int64),
    'net' : Column(pa.Int64),
    'original_title': Column(pa.String), 
    'director': Column(pa.String),
    'runtime': Column(pa.Int64),
    'action': Column(pa.Int),
    'adventure': Column(pa.Int),
    'animation': Column(pa.Int),
    'comedy': Column(pa.Int),
    'crime': Column(pa.Int),
    'documentary': Column(pa.Int),
    'drama': Column(pa.Int),
    'family': Column(pa.Int),
    'fantasy': Column(pa.Int),
    'foreign': Column(pa.Int),
    'history': Column(pa.Int),
    'horror': Column(pa.Int),
    'music': Column(pa.Int),
    'mystery': Column(pa.Int),
    'romance': Column(pa.Int),
    'science_fiction': Column(pa.Int),
    'tv_movie': Column(pa.Int),
    'thriller': Column(pa.Int),
    'war': Column(pa.Int),
    'western': Column(pa.Int),
    'no_genre' : Column(pa.Int),
    'production_companies': Column(pa.String),
    'release_date': Column(pa.DateTime),
    'release_day' : Column(pa.String),
    'holiday' : Column(pa.Bool),
    'vote_count': Column(pa.Int64),
    'vote_average': Column(pa.Float64),
    'release_year': Column(pa.Int64),
    'release_year_groups' : Column(pa.Category),
    'budget_adj' : Column(pa.Float64),
    'revenue_adj' : Column(pa.Float64),
    'net_adj' : Column(pa.Float64)
}

)

In [ ]:
# Put Schema into YAML file
yaml_sche = schema.to_yaml()
f = Path('file.yml')
f.touch()
f.write_text(yaml_sche)

In [ ]:
# open the yaml file
with open('file.yml', 'r') as f:
    file = yaml.safe_load(f)

In [ ]:
# check for duplicates
mdf[mdf.duplicated()]

In [ ]:
# drop dulicates
mdf.drop_duplicates()

In [ ]:
# change NaN to Independent in production companies column
mdf['production_companies']=mdf['production_companies'].replace(np.nan,'Independent')
mdf.info()

In [ ]:
# run column validation
col_val(mdf)

In [ ]:
# Remove columns from movies dataframe that will not be used in analysis and is not in the schema
# leave genres in for now
mdf= mdf.drop(['budget', 'revenue', 'overview', 'imdb_id', 'tagline', 'id', 'homepage', 'cast', 'keywords'], axis=1)     

In [ ]:
mdf.info()

In [ ]:
# check columns
col_val(mdf)

Leave genres for now, begin adding columns for analysis.

In [ ]:
# Add columns to movies dataframe that are in the schema
mdf['net_adj'] = mdf['revenue_adj']-mdf['budget_adj']

Separating Categories from genres columns

- separate categories in genres
- create new columns for genres using 0, 1

In [ ]:
# replace | with ,
mdf['genres']=replace_pipe(mdf, 'genres')

In [ ]:
# List of unique genres
u_g=get_unique_list(mdf, 'genres')
len(u_g)

In [ ]:
# add new genre columns to mdf, set all values to zero
mdf=insert_zero(mdf, u_g)

In [ ]:
# Change new column value to 1 if category is present in old column. 
for g in u_g:
    category_present(mdf, 'genres', g)

In [ ]:
# Schema check
%run validate.py
col_val(mdf)

In [ ]:
# drop old genres
mdf.drop(['genres'], axis=1, inplace=True)

In [ ]:
# rename nan
mdf.rename(columns = {'nan':'no_genre'}, inplace = True)

In [ ]:
# Infor check
mdf.info()

In [ ]:
# column name check
col_val(mdf)

### Clean-Shape Dates 
- Change release_date to datetime
- Add release days and holidays columns
- Get 5 year categories for release years

In [ ]:
# Fixing dates in release date column to yyyy-mm-dd
# split of the values in release_day column to 3 seperate columns
date_fix=mdf['release_date'].str.split('/', expand=True)

# 0-month, 1-day, 2-year
# fill zeros for month and day
date_fix[0]=date_fix[0].str.zfill(2)
date_fix[1]=date_fix[1].str.zfill(2)

In [ ]:
# Fix yyyy format column
date_fix[2]=date_fix[2].astype(int)
# iterate over value and create yyyy
for k, v in date_fix[2].iteritems():
    if v in range(0, 22):
        date_fix[2].iloc[k]=v+2000
    else:
        date_fix[2].iloc[k]=v+1900
       
date_fix[2]=date_fix[2].astype(str)

In [ ]:
# bring the dates back together
date_fix['new']= date_fix[2]+date_fix[0]+date_fix[1]
# view one of the rows
date_fix['new'][9]

In [ ]:
# change new column to datetime format
date_new=pd.to_datetime(date_fix['new'])
date_new.info

In [ ]:
# replace release date column in new datetime format
mdf['release_date']=date_new
mdf['release_date']

In [ ]:
# dataframe check
mdf.info()

In [ ]:
# drop null values for director
# Drop rows if value in director column is null
mdf=mdf.dropna(subset=['director'])
mdf.info()

In [ ]:
col_val(mdf)

In [ ]:
# run datatype validation
dtype_val(mdf)

In [ ]:
# New release day column, indicates day of the week the movie was released
mdf['release_day']=pd.to_datetime(mdf['release_date']).dt.day_name()

In [ ]:
# Create holiday column to indicate if the release day was a holiday
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal=calendar()
holidays=cal.holidays(start=mdf['release_date'].min(), end=mdf['release_date'].max())
mdf['holiday'] = mdf['release_date'].isin(holidays)

In [ ]:
# just curious, checkout holiday count
mdf['holiday'].value_counts()

In [ ]:
# Add release year grouping, evey 5 years
mdf['release_year_groups'] = pd.cut(mdf['release_year'], 11)
mdf['release_year_groups']

In [ ]:
# columns check
col_val(mdf)

In [ ]:
# run datatype validation
dtype_val(mdf)

In [ ]:
# put reshaped dataframe into out.csv
mdf.to_csv('out.csv')

In [ ]:
# read new out csv file, view columns
out=pd.read_csv('out.csv', index_col=[0])
out.columns

## Visualizing  and Analyzing the Movies Dataset

- What 5 year period were the most films released?
- Are some genres more popular per 5 years? 
- What is the most common day of the week that a movie is released? 
- Do movies generate more revenue if they are released certain days or on a holiday?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%run group.py
df=pd.read_csv('out.csv')

Most movies are released in 2010-2015

In [ ]:
# View plot of release year count
df['release_year'].value_counts().plot(title='Number of Films Released between 1959-2015');

2010-2015 had the most released films, 3099, for this dataset that spans 56 years.

In [ ]:
df['release_year_groups'].value_counts()

In [ ]:
df['release_year_groups'].value_counts().plot.pie(title='Film % totals: 5 Year Groups',figsize=(7, 7),autopct='%1.1f%%', label='');

### Movie Genre Analysis
Note: All movie genres, per movie, were included with this analysis.

In [ ]:
# get lists of unique genres and unique 5 year groups
genres_list=['action','adventure','animation','comedy','crime','documentary','drama','family','fantasy','foreign','history','horror','music','mystery','romance','science_fiction','tv_movie','thriller','war','western']
release_year_groups_list=df['release_year_groups'].unique().tolist()
release_year_groups_list

### Genre-year category analysis

In [ ]:
# get function to make year-genre dataframes
def dict(yrgp):
    d=[]
    for g in genres_list:
        sum=df.loc[df['release_year_groups']==yrgp][g].sum()
        d.append({
            'year category': yrgp, 'genre': g, 'sums': sum
        })
    return pd.DataFrame(d)
    

In [ ]:
# make a list of dataframes
dlist = [ dict(yrgp) for yrgp in release_year_groups_list]


### Drama was the most common type of genre film made in all year categories except for 1985-1990, which was Comedy.

In [ ]:
# Genreate bar charts for most common genre per year category
i=0
for d in dlist:
    title=dlist[i]['year category'][1]
    max=d['genre'][d['sums'].idxmax()]
    dlist[i].plot(kind='bar', x='genre', y='sums', title=title);
    print(f'{title} most common genre: {max}')
    i=i+1

### Finding Genre best appearance per year

In [ ]:
# max number of movies released per genre 
row_list=[max_genre_by_year(g) for g in genres_list]
genre_max=pd.DataFrame(row_list, columns=['genres', 'max_released', 'release year'])

Year Categories 2010-2015, 2005-2010, 1965-1970 are where we find genre best appearance years.

In [ ]:
genre_max_s=genre_max.groupby(['release year']).size()
genre_max_s.plot.pie(title='Max Movies made per Genre/5 Years', label='');

- 14 Genres had most appearacnes between 2010-2015

In [ ]:
genre_max.loc[genre_max['release year']=='(2010.0, 2015.0]']

- 5 genres, Family, Fantasy, History and Romance Genres had their best showing between 2005-2010

In [ ]:
genre_max.loc[genre_max['release year']=='(2005.0, 2010.0]']

- Western Movie Genre had it's best showing between 1965-1970

In [ ]:
genre_max.loc[genre_max['release year']=='(1965.0, 1970.0]']

## Days of Week Released and Adjusted Revenue
- Friday is the most popular day for a movie to be released.
- Movies released on Wednesday have highest average adjusted revenue.

In [ ]:
# Get number of films released, by day of the week.
df.groupby(['release_day']).size().plot(kind='bar', title='Number of Movies Released per Day of the Week');

In [ ]:
df.groupby(['release_day'])['revenue_adj'].mean()

### Movies released on Wednesday have highest average adjusted revenue.

In [ ]:
# Plot Average Adjusted Revenueper Day of Week - Movie Release
df.groupby(['release_day'])['revenue_adj'].mean().plot(kind='bar', title='Average Adjusted Revenue per Day of the Week');

### Highest Adjusted Revenue Movie

In [ ]:
df.loc[df['revenue_adj'].idxmax(), ['original_title']]

## Holidays
- Most Holidays a movie was released land on Monday
- Highest Adjusted Revenue Holiday-days are Tuesdays

In [ ]:
# Display Days of Week totlals, if it's a Holiday
holiday=df[df['holiday']==True]
holiday.groupby(['release_day']).size().plot(kind='bar', title='Holiday count by Week day');

In [ ]:
# Determine Ave Revenue Generated on Holidays
holiday.groupby(['release_day'])['revenue_adj'].mean()

### Highest Adjusted Revenue Holiday-days are Tuesdays

In [ ]:
holiday.groupby(['release_day'])['revenue_adj'].mean().plot(kind='bar', title='Holiday Release Day Average Adjusted Revenue');

### Conclusion
- Most Films were released during the 2010-2015 years for this dataset.
- Year Categories 2010-2015, 2005-2010, 1965-1970 are where we find genre best appearance years.
- Western movies had the highest showing during 1965-9170
- Drama was the most common type of genre film made in all year categories except for 1985-1990, which was Comedy. 
- Friday is the most common day of the week that a movie is released 
- Movies generate more revenue if they are released on a Wednesday, according to this dataset.
- Avatar was the highest Adjusted Revenue for this movie dataset.

Limitations:
- Even though I made some functions to deal with the categories within categories for genres, it was difficult.
- Because some columns had multiple categories within one category, I decided to focus on genres.
- Some columns, cast, keywords, homepage, etc...had quite a bit of missing data, so I chose to not include it in the analysis.
- I imagine revenue is skewed due to some movies showing zero dollars.
- Changing the release date to datetime was challenging, splitting the columns, adding zeros and putting it all back together again.

Final Thoughts:
- I was surprised that the most common movie genre is Drama
- Movies released on Wednesday have highest average adjusted revenue was surprising too.
- I enjoy data collection, data cleaning, wrangling and validation.
- I would like to learn more about working with schemas.
- I enjoy working with categorical data, and is why I chose the movie dataset.